# 🔍 CICIDS2017 - Dataset Pré-Nettoyé

**Dataset** : `/kaggle/input/cicids2017-cleaned-and-preprocessed`

Ce notebook traite la version **pré-nettoyée** du CICIDS2017 avec 7 catégories d'attaques.

---

## 📦 Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')


## 📂 Charger le Dataset

In [ ]:
# Charger le dataset pré-nettoyé
df = pd.read_csv('/kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv')

print(f"Shape: {df.shape}")
print(f"Colonnes: {len(df.columns)}")
print(f"Taille en mémoire: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 🔍 Exploration Initiale

In [ ]:
# Premières lignes
df.head()

In [ ]:
# Informations sur les colonnes
df.info()

## 🏷️ Analyse des Labels

In [ ]:
# Distribution des labels
print("Distribution des types d'attaques:")
print(df['Attack Type'].value_counts())
print(f"\nPourcentages:")
print(df['Attack Type'].value_counts(normalize=True) * 100)

In [ ]:
# Vérifier les valeurs uniques (avec longueur pour détecter espaces)
print("Labels uniques:")
for label in df['Attack Type'].unique():
    count = (df['Attack Type'] == label).sum()
    print(f"  '{label}' (len={len(label)}) : {count:,} instances")

In [ ]:
# Visualisation
plt.figure(figsize=(14, 6))
df['Attack Type'].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Distribution des Types d\'Attaques - CICIDS2017 Pré-Nettoyé', fontsize=16, fontweight='bold')
plt.xlabel('Type d\'Attaque', fontsize=12)
plt.ylabel('Nombre d\'Instances', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('attack_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 🧹 Vérification de la Qualité

In [ ]:
# Vérifier les valeurs manquantes
print("Valeurs manquantes:")
nan_counts = df.isnull().sum()
if nan_counts.sum() > 0:
    print(nan_counts[nan_counts > 0])
else:
    print("✅ Aucune valeur NaN")

In [ ]:
# Vérifier les valeurs infinies
print("Valeurs infinies:")
numeric_cols = df.select_dtypes(include=[np.number]).columns
inf_counts = {}
for col in numeric_cols:
    inf_count = np.isinf(df[col]).sum()
    if inf_count > 0:
        inf_counts[col] = inf_count

if inf_counts:
    for col, count in sorted(inf_counts.items(), key=lambda x: x[1], reverse=True)[:5]:
        print(f"  {col}: {count:,}")
else:
    print("✅ Aucune valeur infinie")

In [ ]:
# Vérifier les duplications
dup_count = df.duplicated().sum()
print(f"Duplications: {dup_count:,} ({dup_count/len(df)*100:.2f}%)")

## 🔧 Nettoyage (si nécessaire)

In [ ]:
# Nettoyer si nécessaire
df_clean = df.copy()

# Remplacer infinis par NaN
df_clean.replace([np.inf, -np.inf], np.nan, inplace=True)

# Remplir NaN avec médiane
for col in df_clean.select_dtypes(include=[np.number]).columns:
    if df_clean[col].isnull().any():
        df_clean[col].fillna(df_clean[col].median(), inplace=True)

# Supprimer duplications
df_clean = df_clean.drop_duplicates()

print(f"✅ Nettoyage terminé")
print(f"Shape finale: {df_clean.shape}")

## 🏷️ Stratégie 1 : Classification Binaire

In [ ]:
# Créer labels binaires : 0 = Normal, 1 = Attaque
df_clean['Binary_Label'] = (df_clean['Attack Type'] != 'Normal Traffic').astype(int)

print("Distribution binaire:")
print(df_clean['Binary_Label'].value_counts())
print(f"\nPourcentages:")
print(df_clean['Binary_Label'].value_counts(normalize=True) * 100)

## 🏷️ Stratégie 2 : Classification Multi-Classes (6 catégories)

In [ ]:
# Fusionner DoS et DDoS
df_clean['Attack_Type_Merged'] = df_clean['Attack Type'].replace({
    'DoS': 'DoS_DDoS',
    'DDoS': 'DoS_DDoS'
})

print("Distribution après fusion DoS/DDoS:")
print(df_clean['Attack_Type_Merged'].value_counts())

In [ ]:
# Encoder les labels
le = LabelEncoder()
df_clean['Label_Encoded'] = le.fit_transform(df_clean['Attack_Type_Merged'])

# Afficher le mapping
print("Mapping numérique:")
for i, label in enumerate(le.classes_):
    count = (df_clean['Label_Encoded'] == i).sum()
    print(f"{i}: {label} ({count:,} instances)")

## 🎯 Préparation des Features

In [ ]:
# Séparer features et labels
label_cols = ['Attack Type', 'Attack_Type_Merged', 'Label_Encoded', 'Binary_Label']
feature_cols = [col for col in df_clean.columns if col not in label_cols]

X = df_clean[feature_cols]
y = df_clean['Label_Encoded']  # Ou 'Binary_Label' pour classification binaire

print(f"Features shape: {X.shape}")
print(f"Labels shape: {y.shape}")

In [ ]:
# Normaliser les features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"✅ Features normalisées")
print(f"Shape: {X_scaled.shape}")

## 💾 Sauvegarder les Résultats

In [ ]:
# Sauvegarder le dataset complet
df_clean.to_csv('cicids2017_final.csv', index=False)
print("✅ Dataset complet sauvegardé: cicids2017_final.csv")

In [ ]:
# Sauvegarder X et y pour ML
np.save('X_scaled.npy', X_scaled)
np.save('y.npy', y.values)
print("✅ Features et labels sauvegardés: X_scaled.npy, y.npy")

In [ ]:
# Sauvegarder les encodeurs
import joblib

joblib.dump(le, 'label_encoder.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("✅ Encodeurs sauvegardés: label_encoder.pkl, scaler.pkl")

## 📊 Statistiques Finales

In [ ]:
print("=" * 70)
print("RÉSUMÉ DU PREPROCESSING")
print("=" * 70)
print(f"\nDataset final:")
print(f"  - Lignes: {df_clean.shape[0]:,}")
print(f"  - Features: {X.shape[1]}")
print(f"  - Classes: {len(le.classes_)}")
print(f"\nDistribution des classes:")
for i, label in enumerate(le.classes_):
    count = (y == i).sum()
    pct = (count / len(y)) * 100
    print(f"  {i}: {label:20s} {count:8,} ({pct:5.2f}%)")
print("\n" + "=" * 70)
print("✅ Preprocessing terminé! Prêt pour l'entraînement.")
print("=" * 70)

## 📥 Prochaines Étapes

1. ✅ Dataset exploré et nettoyé
2. ✅ Labels encodés (binaire ou multi-classes)
3. ✅ Features normalisées
4. ✅ Fichiers sauvegardés

**À faire ensuite :**
- Télécharger les fichiers depuis Kaggle Output
- Placer dans `data/processed/` de votre projet local
- Commit sur Git
- Passer au Training ML/DL !

---

**Bon entraînement ! 🚀**